<a href="https://colab.research.google.com/github/hellocybernetics/TensorFlow_Eager_Execution_Tutorials/blob/master/tutorials/intermediate/residual_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
tfk = tf.keras
tfe = tf.contrib.eager

tf.enable_eager_execution()

In [3]:
# Hyper parameters
num_epochs = 10
num_classes = 10
batch_size = 256
learning_rate = 0.001

(x_train, y_train), (x_test, y_test) = tfk.datasets.cifar10.load_data()

print("training_data\n", x_train.shape)
print("test_data\n", x_test.shape)
print("training_label\n", y_train.shape)
print("test_label\n", y_test.shape)

170500096/170498071 [==============================] - 46s 0us/step
training_data
 (50000, 32, 32, 3)
test_data
 (10000, 32, 32, 3)
training_label
 (50000, 1)
test_label
 (10000, 1)


In [4]:
x_train_ = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train_ = tf.reshape(tf.one_hot(y_train, 10), (-1, 10))


print(x_train_.shape)
print(y_train_.shape)

(50000, 32, 32, 3)
(50000, 10)


In [5]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .batch(batch_size)
    .shuffle(10000)
)

train_dataset = (
    train_dataset.map(lambda x, y: 
                      (tf.div(tf.cast(x, tf.float32), 255.0), 
                       tf.reshape(tf.one_hot(y, 10), (-1, 10))))
)

train_dataset = train_dataset.repeat()
print(train_dataset)

<RepeatDataset shapes: ((?, 32, 32, 3), (?, 10)), types: (tf.float32, tf.float32)>


In [6]:
test_dataset = (
    tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .batch(1000)
    .shuffle(10000)
)
test_dataset = (
    test_dataset.map(lambda x, y: 
                      (tf.div(tf.cast(x, tf.float32), 255.0), 
                       tf.reshape(tf.one_hot(y, 10), (-1, 10))))
)
test_dataset = test_dataset.repeat()
print(test_dataset)

<RepeatDataset shapes: ((?, 32, 32, 3), (?, 10)), types: (tf.float32, tf.float32)>


In [0]:
def conv3x3(out_channels, strides=1):
    return tfk.layers.Conv2D(out_channels, kernel_size=3, 
                             strides=strides, padding='same', use_bias=False)

In [0]:
# Residual block
class ResidualBlock(tfk.Model):
    def __init__(self, out_channels, strides=1, downsample=None):
        super(ResidualBlock, self).__init__(name='ResidualBlock')
        self.conv1 = conv3x3(out_channels, strides)
        self.bn1 = tfk.layers.BatchNormalization(axis=-1)
        self.relu = tfk.layers.ReLU()
        self.conv2 = conv3x3(out_channels)
        self.bn2 = tfk.layers.BatchNormalization(axis=-1)
        self.downsample = downsample
        
    def call(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [0]:
class ResNet(tfk.Model):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__(name='ResNet')
        self.in_channels = 16
        self.conv = conv3x3(16)
        self.bn = tfk.layers.BatchNormalization(axis=-1)
        self.relu = tfk.layers.ReLU()
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = tfk.layers.AvgPool2D(8)
        self.flatten = tfk.layers.Flatten()
        self.fc = tfk.layers.Dense(num_classes)
        
    def make_layer(self, block, out_channels, blocks, strides=1):
        downsample = None
        if (strides != 1) or (self.in_channels != out_channels):
            downsample = tfk.Sequential([
                conv3x3(out_channels, strides=strides),
                tfk.layers.BatchNormalization(axis=-1)])
        layers = []
        layers.append(block(out_channels, strides, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels))
        return tfk.Sequential(layers)
    
    def call(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = self.flatten(out)
        out = self.fc(out)
        return out

In [0]:
model = ResNet(ResidualBlock, [2, 2, 2])

In [11]:
model.compile(optimizer=tf.train.AdamOptimizer(learning_rate),
              loss=tf.losses.softmax_cross_entropy,
              metrics=["accuracy"])
model(x_train_[:50]).shape
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  432       
_________________________________________________________________
batch_normalization (BatchNo multiple                  64        
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
sequential (Sequential)      multiple                  9472      
_________________________________________________________________
sequential_2 (Sequential)    multiple                  37504     
_________________________________________________________________
sequential_4 (Sequential)    multiple                  148736    
_________________________________________________________________
average_pooling2d (AveragePo multiple                  0         
__________

In [12]:
model.fit(train_dataset,
          batch_size=batch_size,
          epochs=num_epochs,
          steps_per_epoch=int(x_train.shape[0]/batch_size),
)

Epoch 1/10
195/195 [==============================] - 34s 176ms/step - loss: 1.4596 - acc: 0.4686
Epoch 2/10
195/195 [==============================] - 34s 172ms/step - loss: 1.0619 - acc: 0.6191
Epoch 3/10
195/195 [==============================] - 33s 172ms/step - loss: 0.8769 - acc: 0.6908
Epoch 4/10
195/195 [==============================] - 34s 172ms/step - loss: 0.7570 - acc: 0.7326
Epoch 5/10
195/195 [==============================] - 33s 171ms/step - loss: 0.6612 - acc: 0.7683
Epoch 6/10
195/195 [==============================] - 33s 172ms/step - loss: 0.5843 - acc: 0.7966
Epoch 7/10
195/195 [==============================] - 33s 171ms/step - loss: 0.5179 - acc: 0.8201
Epoch 8/10
195/195 [==============================] - 34s 173ms/step - loss: 0.4656 - acc: 0.8364
Epoch 9/10
195/195 [==============================] - 33s 172ms/step - loss: 0.4131 - acc: 0.8549
Epoch 10/10
195/195 [==============================] - 33s 172ms/step - loss: 0.3726 - acc: 0.8712


In [17]:
model.evaluate(test_dataset, steps=100)

100/100 [==============================] - 14s 142ms/step


[0.7618808209896087, 0.7497999966144562]